In [1]:
import pandas as pd
import numpy as np
import bz2
import json
import findspark
findspark.init('/Users/tatianacogne/spark')
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from datetime import datetime
import plotly.express as px
from datetime import timedelta as dt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.mode.chained_assignment = None

#### Dict and DF useful for this notebook

In [2]:
from_qids_to_label_occupation = pd.read_json('data/occupations.json') 
from_qids_to_label_ethnics = pd.read_json('data/ethnic.json' ) 
from_qids_to_label_religion = pd.read_json('data/religion.json') 
from_qids_to_label_nationality = pd.read_json('data/nationality.json') 
from_qids_to_label_party = pd.read_json('data/party.json') 
from_qids_to_label_gender = pd.read_json('data/gender.json') 

df_speakers_attributes = pd.read_parquet('data/speaker_attributes.parquet')

In [3]:
id_musk = df_speakers_attributes[df_speakers_attributes.label =='Elon Musk'].id.values[0]

### Process for the speaker attributs

In [4]:
def map_qids_labels(list_qids,df_):
    """Transform a list of QIDS into a list of labels, return 0 if the list is empty (which is equal to 0 in the dataframe)."""
    if(type(list_qids)!=int):
        return [df_[df_.qids == x].values[0][1] for x in list_qids if x!=0 and len(df_[df_.qids == x].values)>=1]    
    else:
        return 0  

In [5]:
def process_speakers_attributes(df_speakers_attributes):
    """Add the columns for the party, occupation, religion, ethnics and nationality their labels. Take as input the dataframe with the selected speakers and return the same dataframe with the columns added."""
    df = df_speakers_attributes[['date_of_birth','nationality','gender','ethnic_group','occupation','id','label','religion','party']]
    df = df.rename(columns={'date_of_birth':'birth','nationality':'nationality_qids','gender':'gender_qids','ethnic_group':'ethnics_qids','occupation':'occupations_qids','id':'name_qids','label':'name_label','religion':'religion_qids','party':'party_qids','gender':'gender_qids'})
    # Add labels for occupations, ethincs, religion, nationality, party
    df = df.fillna(0)
    df['occupation_label']= df['occupations_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_occupation))
    df['ethnics_label']= df['ethnics_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_ethnics))
    df['religion_label']= df['religion_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_religion))
    df['nationality_label']= df['nationality_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_nationality))
    df['party_label']= df['party_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_party))
    df['gender_label']= df['gender_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_gender))
    return df
    

#### Select only the QIDS needed in the analysis

In [6]:
def select_speakers(list_qids):
    """From a list of QIDS of speakers, return the dataframe with only the speakers that are in the list."""
    return df_speakers_attributes[df_speakers_attributes.id.isin(list_qids)]

### Process for the quotes dataframe

In [36]:
def process_dataframe_quotes(df_sent, speakers):
    """ Transform the date into a datetime object ('%Y-%m-%d %H:%M:%S). Join the quotes dataframe with the speakers dataframe on the name_qids."""
    
    # Rename the columns and keep only the one needed
    df_1 = df_sent[['name_qids','quotation','quote_qids','review','Vader_Sentiment']]
    
    # Change timestamp in datetime object and add the perdio depending on the time_range list
    df_sent['date']= df_sent['date'].apply(lambda x :datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))
    df_sent['year_month']= df_sent['date'].apply(lambda x :datetime.strptime(str(x)[:7], '%Y-%m'))
    
    #Join the dataframe with the speakers attributes on the name_qids
    df = df_sent.merge(speakers, on='name_qids')
    return df
  

#### Chevrolet

In [37]:
df_chevrolet= pd.read_json('data/chevrolet_sentiment.json')

list_qids_chevrolet = df_chevrolet.drop_duplicates(['name_qids']).name_qids.values
df = select_speakers(list_qids_chevrolet)
df_speakers_att_chevrolet = process_speakers_attributes(df)

speakers = df_speakers_att_chevrolet[['name_qids','name_label','occupations_qids']]
df_chevrolet = process_dataframe_quotes(df_chevrolet, speakers)
df_chevrolet.head(3)

,date,name_qids,quotation,quote_qids,Vader_Sentiment,emotion,review,year_month,name_label_x,occupations_qids_x,occupation_label,nationality_label,name_label_y,occupations_qids_y
0,2015-02-04 05:35:55,Q5178606,I've had to go back 10 years of my program fin...,53298,0.0000,Surprise,5,2015-02-01,Courtney Force,[Q378622],[racing driver],[United States of America],Courtney Force,[Q378622]
1,2015-10-31 09:27:41,Q5178606,Robert Hight stepped up with Mike Neff and tha...,50268,0.0000,Happy,5,2015-10-01,Courtney Force,[Q378622],[racing driver],[United States of America],Courtney Force,[Q378622]
2,2015-07-25 11:08:06,Q5178606,I saw (crew chief Jon) Schaffer running for th...,21150,0.9834,Happy,4,2015-07-01,Courtney Force,[Q378622],[racing driver],[United States of America],Courtney Force,[Q378622]


#### Tesla

In [9]:
df_tesla= pd.read_json('data/tesla_sentiment.json')
df_tesla.head(3)

list_qids_tesla = [x for x in df_tesla.drop_duplicates(['name_qids']).name_qids.values if x!= id_musk]

df = select_speakers(list_qids_tesla)
df_speakers_att_tesla = process_speakers_attributes(df)
df_speakers_att_tesla.head(3)

speakers = df_speakers_att_tesla[['name_qids','name_label','occupations_qids']]
df_tesla = process_dataframe_quotes(df_tesla, speakers)
df_tesla.head(3)

,date,name_qids,quotation,quote_qids,Vader_Sentiment,emotion,review,year_month,name_label_x,occupations_qids_x,occupation_label,nationality_label,name_label_y,occupations_qids_y
0,2015-04-17 17:55:00,Q6751189,The policy will concentrate on `A' category st...,76279,0.0000,Fear,4,2015-04-01,Manoj Sinha,[Q82955],[politician],[India],Manoj Sinha,[Q82955]
1,2015-10-29 16:31:31,Q4934,As Sergey and I wrote in the original founders...,10100,0.6908,Surprise,4,2015-10-01,Larry Page,"[Q131524, Q82594, Q81096]","[entrepreneur, computer scientist, engineer]",[United States of America],Larry Page,"[Q131524, Q82594, Q81096]"
2,2015-04-20 23:38:36,Q4934,"In the first week of March 2013, Musk reached ...",30429,0.5423,Fear,1,2015-04-01,Larry Page,"[Q131524, Q82594, Q81096]","[entrepreneur, computer scientist, engineer]",[United States of America],Larry Page,"[Q131524, Q82594, Q81096]"


### General Plot

In [38]:
df_chevrolet_plot = df_chevrolet.groupby(by='year_month').mean().reset_index()
df_chevrolet_plot_review = df_chevrolet_plot[['year_month','Vader_Sentiment']]
df_chevrolet_plot_review['count_'] = df_chevrolet.groupby(by='year_month').count().reset_index()['date']

fig = make_subplots(rows=2, cols=1, row_heights=[2, 1],vertical_spacing=0.1)
fig.add_trace(go.Bar(x=df_chevrolet_plot_review.year_month, y=df_chevrolet_plot_review.count_,),row=1, col=1)
fig.add_trace(go.Scatter(x=df_chevrolet_plot_review.year_month, y=df_chevrolet_plot_review.Vader_Sentiment),row=2, col=1)
fig.show()

In [39]:
df_tesla_plot = df_tesla.groupby(by='year_month').mean().reset_index()
df_tesla_plot_review = df_tesla_plot[['year_month','Vader_Sentiment']]
df_tesla_plot_review['count_'] = df_tesla.groupby(by='year_month').count().reset_index()['date']

fig = make_subplots(rows=2, cols=1, row_heights=[2, 1],vertical_spacing=0.1)
fig.add_trace(go.Bar(x=df_tesla_plot_review.year_month, y=df_tesla_plot_review.count_,),row=1, col=1)
fig.add_trace(go.Scatter(x=df_tesla_plot_review.year_month, y=df_tesla_plot_review.Vader_Sentiment),row=2, col=1)
fig.show()

In [40]:
def find_review_event(sortie, df, n_day,num_day_event):
    df['year_month_day']= df['date'].apply(lambda x :datetime.strptime(str(x)[:10], '%Y-%m-%d'))
    start_event = sortie - dt(days=num_day_event)
    end_event = sortie + dt(days=num_day_event)
    n_days_ago = sortie - dt(days=n_day+num_day_event)
    n_days_after = sortie + dt(days=n_day+num_day_event)
    
    df_event = df[(df.year_month_day <= end_event) & (df.year_month_day >= start_event)]
    df_event['type'] = ['during']*df_event.shape[0]
    df_before_event = df[(df.year_month_day < start_event) & (df.year_month_day > n_days_ago)]
    df_before_event['type'] = ['before']*df_before_event.shape[0]
    df_after_event = df[(df.year_month_day < n_days_after) & (df.year_month_day > end_event)]
    df_after_event['type'] = ['after']*df_after_event.shape[0]
    
    return pd.concat([df_before_event, df_event, df_after_event])

### BoxPlot Event (before, during, after)

In [45]:
# 
sortie_chev_1 = datetime.strptime(str('2018-12-16'), '%Y-%m-%d')
n_day = 20
num_day_event = 10
df  = find_review_event(sortie_chev_1, df_chevrolet, n_day, num_day_event)

fig = px.box(df, x="type", y='Vader_Sentiment')
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [49]:
# When was Tesla Enter Power Blabla released
sortie_tesla_1 = datetime.strptime(str('2015-04-30'), '%Y-%m-%d')
n_day = 20
num_day_event = 10
df  = find_review_event(sortie_tesla_1, df_tesla, n_day, num_day_event)

fig = px.box(df, x="type", y='Vader_Sentiment')
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [46]:
# When was Tesla Model 3 released
sortie_tesla_1 = datetime.strptime(str('2017-07-07'), '%Y-%m-%d')
n_day = 20
num_day_event = 10
df  = find_review_event(sortie_tesla_1, df_tesla, n_day, num_day_event)

fig = px.box(df, x="type", y='Vader_Sentiment')
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [47]:
# When was Tesla Model Y released
sortie_tesla_1 = datetime.strptime(str('2020-03-13'), '%Y-%m-%d')
n_day = 20
num_day_event = 10
df  = find_review_event(sortie_tesla_1, df_tesla, n_day, num_day_event)

fig = px.box(df, x="type", y='Vader_Sentiment')
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

### PieChart
#### Occupations 

In [99]:
# Tesla
df_t = df_tesla.explode('occupation_label').groupby(by='occupation_label').count().reset_index()
df_t.loc[df_t['date'] < 150, 'occupation_label'] = 'Others' 
# Chevrolet 
df_c = df_chevrolet.explode('occupation_label').groupby(by='occupation_label').count().reset_index()
df_c.loc[df_c['date'] < 100, 'occupation_label'] = 'Others'
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(go.Pie(labels=df_t.occupation_label, values=df_t.date),row=1, col=1)
fig.add_trace(go.Pie(labels=df_c.occupation_label, values=df_c.date),row=1, col=2)
fig.show()

#### Nationality

In [102]:
# Tesla
df_t = df_tesla.explode('nationality_label').groupby(by='nationality_label').count().reset_index()
df_t.loc[df_t['date'] < 150, 'nationality_label'] = 'Others' 
# Chevrolet 
df_c = df_chevrolet.explode('nationality_label').groupby(by='nationality_label').count().reset_index()
df_c.loc[df_c['date'] < 100, 'nationality_label'] = 'Others'
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(go.Pie(labels=df_t.nationality_label, values=df_t.date),row=1, col=1)
fig.add_trace(go.Pie(labels=df_c.nationality_label, values=df_c.date),row=1, col=2)
fig.show()

In [90]:
# Chevrolet 
df = df_chevrolet.explode('occupation_label').groupby(by='occupation_label').count().reset_index()
df.loc[df['date'] < 100, 'occupation_label'] = 'Others' # Represent only large countries

fig = px.pie(df, values='date', names='occupation_label', color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [69]:
import plotly.express as px
df = px.data.gapminder().query("year == 2007").query("continent == 'Europe'")
#df.loc[df['pop'] < 2.e6, 'country'] = 'Other countries' # Represent only large countries
fig = px.pie(df, values='pop', names='country', title='Population of European continent')
fig.show()

In [70]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
83,Austria,Europe,2007,79.829,8199783,36126.492700,AUT,40
119,Belgium,Europe,2007,79.441,10392226,33692.605080,BEL,56
155,Bosnia and Herzegovina,Europe,2007,74.852,4552198,7446.298803,BIH,70
191,Bulgaria,Europe,2007,73.005,7322858,10680.792820,BGR,100
383,Croatia,Europe,2007,75.748,4493312,14619.222720,HRV,191
407,Czech Republic,Europe,2007,76.486,10228744,22833.308510,CZE,203
419,Denmark,Europe,2007,78.332,5468120,35278.418740,DNK,208
527,Finland,Europe,2007,79.313,5238460,33207.084400,FIN,246
539,France,Europe,2007,80.657,61083916,30470.016700,FRA,250
